In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/Contriever/train')

In [3]:
import torch
import numpy as np
import time
import sys
import logging
import os
import src
import transformers
from tqdm import tqdm

In [4]:
from tqdm import TqdmExperimentalWarning
import warnings
warnings.filterwarnings("ignore", category=TqdmExperimentalWarning)

# Set up basic logging configuration

In [5]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

# Preprocessing
- tokenize the corpus_plain.txt file
- store the token ids in tensors and save them to .pt or .pkl files- input for the data loading process in train function

In [6]:
import transformers

def save(tensor, split_path):
    os.makedirs(os.path.dirname(split_path), exist_ok=True)
    with open(split_path, 'wb') as fout:
        torch.save(tensor, fout)

In [7]:
def apply_tokenizer(path, tokenizer):
    alltokens = []
    lines = []
    total_lines = sum(1 for _ in open(path, "r", encoding="utf-8"))
    with open(path, "r", encoding="utf-8") as fin, tqdm(total=total_lines, desc="Tokenizing lines") as pbar:
        for k, line in enumerate(fin):
            lines.append(line.strip())
            pbar.update(1)
            # Batch encode every 1 million lines or so for efficiency
            if len(lines) > 1000000:
                tokens = tokenizer.batch_encode_plus(lines, add_special_tokens=False, truncation=True)['input_ids']
                tokens = [torch.tensor(x, dtype=torch.int) for x in tokens]
                alltokens.extend(tokens)
                lines = []

    # Encode any leftover lines
    if lines:
        tokens = tokenizer.batch_encode_plus(lines, add_special_tokens=False, truncation=True)['input_ids']
        tokens = [torch.tensor(x, dtype=torch.int) for x in tokens]
        alltokens.extend(tokens)

    # Concatenate all token tensors into one large tensor
    alltokens = torch.cat(alltokens)
    return alltokens

In [8]:
pt_file_path = "/content/drive/MyDrive/Contriever/datasets/wikipedia/wiki_tokens.pt"

#load the tensor
tokens = torch.load(pt_file_path, map_location='cpu')

#view basic info
print(f"Total tokens: {tokens.shape[0]}")
print("First 5 sequences (as token IDs):")

#view sequences of length 32
chunk_length = 32
for i in range(5):
    start = i * chunk_length
    end = start + chunk_length
    print(f"\nSequence {i+1}:")
    print(tokens[start:end].tolist())

Total tokens: 46843508
First 5 sequences (as token IDs):

Sequence 1:
[9617, 11140, 2964, 9617, 11140, 2964, 2003, 1037, 2576, 4695, 1998, 2929, 2008, 2003, 8040, 23606, 7476, 1997, 3691, 1998, 19164, 2035, 26097, 1010, 24873, 11890, 3512, 3596, 1997, 12571, 1012, 9617]

Sequence 2:
[11140, 2964, 4455, 2005, 1996, 15766, 1997, 1996, 2110, 1010, 2029, 2009, 4324, 2000, 2022, 14203, 1010, 6151, 2229, 7895, 3468, 1010, 1998, 17631, 1012, 2004, 1037, 7145, 2187, 1011, 3358, 2929]

Sequence 3:
[1010, 2872, 2006, 1996, 2521, 20515, 2187, 1997, 1996, 2576, 8674, 1010, 2009, 2003, 2788, 2649, 4077, 15029, 2964, 1998, 19297, 27255, 2004, 1996, 19297, 3358, 1006, 19297, 14649, 1007, 1997, 1996]

Sequence 4:
[6102, 2929, 1010, 1998, 2038, 1037, 2844, 3439, 2523, 2007, 3424, 1011, 16498, 1998, 14649, 1012, 4286, 2973, 1999, 8384, 2302, 5337, 7632, 6906, 11140, 3111, 2146, 2077, 1996, 5069, 1997, 5337]

Sequence 5:
[2163, 1010, 18814, 1010, 2030, 23560, 1012, 2007, 1996, 4125, 1997, 7362, 25835, 42

# Functions for data loading

In [9]:
def load_data(opt, tokenizer):
    datasets = {}
    for path in opt.train_data:
        data = load_dataset(path, opt.loading_mode)
        if data is not None:
            datasets[path] = Dataset(data, opt.chunk_length, tokenizer, opt)
    dataset = MultiDataset(datasets)
    dataset.set_prob(coeff=opt.sampling_coefficient)
    return dataset

In [10]:
def load_dataset(data_path, loading_mode):
    files = glob.glob(os.path.join(data_path, "*.p*"))
    files.sort()
    tensors = []
    if loading_mode == "split":
        files_split = list(np.array_split(files, dist_utils.get_world_size()))[dist_utils.get_rank()]
        for filepath in files_split:
            try:
                tensors.append(torch.load(filepath, map_location="cpu"))
            except:
                logger.warning(f"Unable to load file {filepath}")
    elif loading_mode == "full":
        for fin in files:
            tensors.append(torch.load(fin, map_location="cpu"))
    elif loading_mode == "single":
        tensors.append(torch.load(files[0], map_location="cpu"))
    if len(tensors) == 0:
        return None
    tensor = torch.cat(tensors)
    return tensor

In [11]:
def deleteword(x, p=0.1):
    mask = np.random.rand(len(x))
    x = [e for e, m in zip(x, mask) if m > p]
    return x


def replaceword(x, min_random, max_random, p=0.1):
    mask = np.random.rand(len(x))
    x = [e if m > p else random.randint(min_random, max_random) for e, m in zip(x, mask)]
    return x


def maskword(x, mask_id, p=0.1):
    mask = np.random.rand(len(x))
    x = [e if m > p else mask_id for e, m in zip(x, mask)]
    return x


def shuffleword(x, p=0.1):
    count = (np.random.rand(len(x)) < p).sum()
    """Shuffles any n number of values in a list"""
    indices_to_shuffle = random.sample(range(len(x)), k=count)
    to_shuffle = [x[i] for i in indices_to_shuffle]
    random.shuffle(to_shuffle)
    for index, value in enumerate(to_shuffle):
        old_index = indices_to_shuffle[index]
        x[old_index] = value
    return x

In [12]:
def apply_augmentation(x, opt):
    if opt.augmentation == "mask":
        return torch.tensor(maskword(x, mask_id=opt.mask_id, p=opt.prob_augmentation))
    elif opt.augmentation == "replace":
        return torch.tensor(
            replaceword(x, min_random=opt.start_id, max_random=opt.vocab_size - 1, p=opt.prob_augmentation)
        )
    elif opt.augmentation == "delete":
        return torch.tensor(deleteword(x, p=opt.prob_augmentation))
    elif opt.augmentation == "shuffle":
        return torch.tensor(shuffleword(x, p=opt.prob_augmentation))
    else:
        if not isinstance(x, torch.Tensor):
            x = torch.Tensor(x)
        return x

In [13]:
def add_bos_eos(x, bos_token_id, eos_token_id):
    if not isinstance(x, torch.Tensor):
        x = torch.Tensor(x)
    if bos_token_id is None and eos_token_id is not None:
        x = torch.cat([x.clone().detach(), torch.tensor([eos_token_id])])
    elif bos_token_id is not None and eos_token_id is None:
        x = torch.cat([torch.tensor([bos_token_id]), x.clone().detach()])
    elif bos_token_id is None and eos_token_id is None:
        pass
    else:
        x = torch.cat([torch.tensor([bos_token_id]), x.clone().detach(), torch.tensor([eos_token_id])])
    return x

In [14]:
def build_mask(tensors):
    shapes = [x.shape for x in tensors]
    maxlength = max([len(x) for x in tensors])
    returnmasks = []
    ids = []
    for k, x in enumerate(tensors):
        returnmasks.append(torch.tensor([1] * len(x) + [0] * (maxlength - len(x))))
        ids.append(torch.cat((x, torch.tensor([0] * (maxlength - len(x))))))
    ids = torch.stack(ids, dim=0).long()
    returnmasks = torch.stack(returnmasks, dim=0).bool()
    return ids, returnmasks


In [15]:
def randomcrop(x, ratio_min, ratio_max):

    ratio = random.uniform(ratio_min, ratio_max)
    length = int(len(x) * ratio)
    start = random.randint(0, len(x) - length)
    end = start + length
    crop = x[start:end].clone()
    return crop

In [16]:
class Dataset(torch.utils.data.Dataset):
    """Monolingual dataset based on a list of paths"""

    def __init__(self, data, chunk_length, tokenizer, opt):

        self.data = data
        self.chunk_length = chunk_length
        self.tokenizer = tokenizer
        self.opt = opt
        self.generate_offset()

    def __len__(self):
        return (self.data.size(0) - self.offset) // self.chunk_length

    def __getitem__(self, index):
        start_idx = self.offset + index * self.chunk_length
        end_idx = start_idx + self.chunk_length
        tokens = self.data[start_idx:end_idx]
        q_tokens = randomcrop(tokens, self.opt.ratio_min, self.opt.ratio_max)
        k_tokens = randomcrop(tokens, self.opt.ratio_min, self.opt.ratio_max)
        q_tokens = apply_augmentation(q_tokens, self.opt)
        q_tokens = add_bos_eos(q_tokens, self.tokenizer.bos_token_id, self.tokenizer.eos_token_id)
        k_tokens = apply_augmentation(k_tokens, self.opt)
        k_tokens = add_bos_eos(k_tokens, self.tokenizer.bos_token_id, self.tokenizer.eos_token_id)

        return {"q_tokens": q_tokens, "k_tokens": k_tokens}

    def generate_offset(self):
        self.offset = random.randint(0, self.chunk_length - 1)

In [17]:
class MultiDataset(torch.utils.data.Dataset):
    def __init__(self, datasets):

        self.datasets = datasets
        self.prob = [1 / len(self.datasets) for _ in self.datasets]
        self.dataset_ids = list(self.datasets.keys())

    def __len__(self):
        return sum([len(dataset) for dataset in self.datasets.values()])

    def __getitem__(self, index):
        dataset_idx = numpy.random.choice(range(len(self.prob)), 1, p=self.prob)[0]
        did = self.dataset_ids[dataset_idx]
        index = random.randint(0, len(self.datasets[did]) - 1)
        sample = self.datasets[did][index]
        sample["dataset_id"] = did
        return sample

    def generate_offset(self):
        for dataset in self.datasets.values():
            dataset.generate_offset()

    def set_prob(self, coeff=0.0):

        prob = np.array([float(len(dataset)) for _, dataset in self.datasets.items()])
        prob /= prob.sum()
        prob = np.array([p**coeff for p in prob])
        prob /= prob.sum()
        self.prob = prob

In [18]:
class Collator(object):
    def __init__(self, opt):
        self.opt = opt

    def __call__(self, batch_examples):

        batch = defaultdict(list)
        for example in batch_examples:
            for k, v in example.items():
                batch[k].append(v)

        q_tokens, q_mask = build_mask(batch["q_tokens"])
        k_tokens, k_mask = build_mask(batch["k_tokens"])

        batch["q_tokens"] = q_tokens
        batch["q_mask"] = q_mask
        batch["k_tokens"] = k_tokens
        batch["k_mask"] = k_mask

        return batch

# Train function

In [19]:
from src import utils, dist_utils
from torch.utils.data import DataLoader, RandomSampler
import numpy
from collections import defaultdict

def train(opt, model, optimizer, scheduler, step):

    run_stats = utils.WeightedAvgStats()  #object to store and average loss,accuracy etc

    tb_logger = utils.init_tb_logger(opt.output_dir)

    logger.info("Data loading")

    if isinstance(model, torch.nn.parallel.DistributedDataParallel):   #get tokenizer from the model
        tokenizer = model.module.tokenizer
    else:
        tokenizer = model.tokenizer
    collator = Collator(opt=opt)
    train_dataset = load_data(opt, tokenizer)  #custom dataset object
    #logger.warning(f"Data loading finished for rank {dist_utils.get_rank()}")

    train_sampler = RandomSampler(train_dataset)  #returns randomly shuffled indices of documents so that in each epoch, the batches contain different set of documents
    train_dataloader = DataLoader(
        train_dataset,
        sampler=train_sampler,
        batch_size=opt.per_gpu_batch_size,
        drop_last=True,
        num_workers=opt.num_workers,
        collate_fn=collator,
    )

    #view train_dataloader
    batch = next(iter(train_dataloader))

    # Print keys and shape of each item in the batch
    print("Batch keys:", batch.keys())
    for key, value in batch.items():
        if isinstance(value, torch.Tensor):
            print(f"{key}: shape = {value.shape}, dtype = {value.dtype}")
        elif isinstance(value, list):
            print(f"{key}: list of length {len(value)}, first item type: {type(value[0])}")
        else:
            print(f"{key}: type = {type(value)}")

    print("\nExample values from batch[0]:")
    for key, value in batch.items():
        try:
            if isinstance(value, torch.Tensor):
                print(f"{key}[0]:", value[0])
            elif isinstance(value, list):
                print(f"{key}[0]:", value[0])
            else:
                print(f"{key}: (non-indexable) {value}")
        except Exception as e:
            print(f"{key}: Error accessing [0] - {e}")

    epoch = 1

    model.train()
    while step < opt.total_steps:
        train_dataset.generate_offset()  #randomizes the starting position(offset) of chunks in each epoch- prevents the model from seeing the same slice of the documents in each epoch

        logger.info(f"Start epoch {epoch}")
        for i, batch in enumerate(train_dataloader): #acc to the random offset generated, each document is cropped, augmented, padded, masked and grouped into batches in train_dataloader now
            step += 1

            batch = {key: value.to(opt.device) if isinstance(value, torch.Tensor) else value for key, value in batch.items()}

            train_loss, iter_stats = model(**batch, stats_prefix="train")  #forward pass (internally calls the forward pass of moco where embeddings for positive pairs are generated, loss is calculated using positive pair embeddings and negative embeddings in the queue

            train_loss.backward()
            optimizer.step()

            scheduler.step()
            model.zero_grad()

            run_stats.update(iter_stats)

            if step % opt.log_freq == 0:
                log = f"{step} / {opt.total_steps}"
                for k, v in sorted(run_stats.average_stats.items()):
                    log += f" | {k}: {v:.3f}"
                    if tb_logger:
                        tb_logger.add_scalar(k, v, step)
                log += f" | lr: {scheduler.get_last_lr()[0]:0.3g}"
                if torch.cuda.is_available():
                    log += f" | Memory: {torch.cuda.max_memory_allocated() // 1e9:.2f} GiB"
                else:
                    log += " | Memory: N/A (CPU mode)"

                logger.info(log)
                run_stats.reset()

            if step % opt.eval_freq == 0:
                if isinstance(model, torch.nn.parallel.DistributedDataParallel):
                    encoder = model.module.get_encoder()
                else:
                    encoder = model.get_encoder()
                eval_model(
                    opt, query_encoder=encoder, doc_encoder=encoder, tokenizer=tokenizer, tb_logger=tb_logger, step=step
                )

                if dist_utils.is_main():
                    utils.save(model, optimizer, scheduler, step, opt, opt.output_dir, f"lastlog")

                model.train()

            if dist_utils.is_main() and step % opt.save_freq == 0:
                utils.save(model, optimizer, scheduler, step, opt, opt.output_dir, f"step-{step}")

            if step > opt.total_steps:
                break
        epoch += 1
    return model


# Eval function

In [20]:
def eval_model(opt, query_encoder, doc_encoder, tokenizer, tb_logger, step):
    print("Evaluating the datasets", opt.eval_datasets)

    for datasetname in opt.eval_datasets:
        metrics = beir_utils.evaluate_model(
            query_encoder,
            doc_encoder,
            tokenizer,
            dataset=datasetname,
            batch_size=opt.per_gpu_eval_batch_size,
            norm_doc=opt.norm_doc,
            norm_query=opt.norm_query,
            beir_dir=opt.eval_datasets_dir,
            score_function=opt.score_function,
            lower_case=opt.lower_case,
            normalize_text=opt.eval_normalize_text,
        )


        message = []
        if dist_utils.is_main():
            for metric in ["NDCG@10", "Recall@10", "Recall@100"]:
                message.append(f"{datasetname}/{metric}: {metrics[metric]:.2f}")
                if tb_logger is not None:
                    tb_logger.add_scalar(f"{datasetname}/{metric}", metrics[metric], step)
            logger.info(" | ".join(message))

# Main

# Instantiate an object of the Options class
- sets up the pipeline for training
- options centralises all required script options
- facilitates the passing of command line arguments
- stores defaults
- prints and saves configuration for future reference

In [21]:
%pip install beir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.0/288.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [ ]:
import src
from src.options import Options
from src import dist_utils, utils, beir_utils
import glob
import random

options= Options()
opt = options.parse() #opt contains a dictionary of all the parsed arguments like seed, batch_size, model_type
opt.train_data = ["/content/drive/MyDrive/Contriever/datasets/wikipedia"]
opt.retriever_model_id = "facebook/contriever"

os.makedirs(opt.output_dir, exist_ok=True) #creates an output directory(path as given in cli) to store the results
os.environ["TOKENIZERS_PARALLELISM"] = "false" #
utils.init_logger(opt) #initialises custom logging, logs to the specified file
torch.manual_seed(opt.seed) #for reproducibility

from src import moco, inbatch
opt.contrastive_mode = "moco"
model_class = moco.MoCo if opt.contrastive_mode == "moco" else inbatch.InBatch #select a learning strategy based on the cli argument

opt.device=torch.device("cuda" if torch.cuda.is_available() else "cpu") #use cpu or gpu if available
model= model_class(opt).to(opt.device) #create an instance of the model (either moco or inbatch
optimizer, scheduler = utils.set_optim(opt, model) #call a function in src.utils that creates an optimiser and learning rate scheduler
step=0 #initialise training step variable

#initialising opt
opt.chunk_length = 256
opt.ratio_min = 0.7
opt.ratio_max = 1.0
opt.augmentation = "mask"
opt.prob_augmentation = 0.3
tokenizer = utils.load_hf(transformers.AutoTokenizer, opt.retriever_model_id)
opt.mask_id = tokenizer.mask_token_id
opt.start_id = 0
opt.vocab_size = tokenizer.vocab_size
opt.num_workers = 0
opt.sampling_coefficient = 0.0

opt.loading_mode = "single"


opt.output_dir = "/content/drive/MyDrive/Contriever/train/output"
opt.total_steps = 400
opt.log_freq = 50
opt.eval_freq = 100
opt.save_freq = 50
opt.queue_size = 65536
opt.momentum = 0.999
opt.temperature = 0.05
opt.label_smoothing = 0.0
opt.norm_doc = True
opt.norm_query = True
opt.pooling = "mean"
opt.random_init = False
opt.moco_train_mode_encoder_k = False
opt.per_gpu_batch_size = 2


opt.eval_datasets=["nq","triviaqa"]
opt.per_gpu_eval_batch_size = 1
opt.eval_datasets_dir = "/content/drive/MyDrive/Contriever/datasets/beir"
opt.score_function = "dot"
opt.lower_case = True
opt.eval_normalize_text = True



model1= train(opt, model, optimizer, scheduler, step)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

  0%|          | 0/11313 [00:00<?, ?it/s]

INFO:beir.datasets.data_loader:Loaded 11313 TEST Documents.
INFO:beir.datasets.data_loader:Doc Example: {'text': 'No context found', 'title': ''}
INFO:beir.datasets.data_loader:Loading Queries...
INFO:beir.datasets.data_loader:Loaded 11313 TEST Queries.
INFO:beir.datasets.data_loader:Query Example: Who was the man behind The Chipmunks?
INFO:beir.retrieval.search.dense.exact_search:Encoding Queries...


Corpus size: 1000
Queries size: 11313


INFO:beir.retrieval.search.dense.exact_search:Sorting Corpus by document length (Longest first)...
INFO:beir.retrieval.search.dense.exact_search:Encoding Corpus in batches... Warning: This might take a while!
INFO:beir.retrieval.search.dense.exact_search:Scoring Function: Dot Product (dot)
INFO:beir.retrieval.search.dense.exact_search:Encoding Batch 1/1...
INFO:beir.retrieval.evaluation:For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
INFO:beir.retrieval.evaluation:

INFO:beir.retrieval.evaluation:NDCG@1: 0.0001
INFO:beir.retrieval.evaluation:NDCG@3: 0.0002
INFO:beir.retrieval.evaluation:NDCG@5: 0.0003
INFO:beir.retrieval.evaluation:NDCG@10: 0.0004
INFO:beir.retrieval.evaluation:NDCG@100: 0.0019
INFO:beir.retrieval.evaluation:NDCG@1000: 0.0109
INFO:beir.retrieval.evaluation:

INFO:beir.retrieval.evaluation:MAP@1: 0.0001
INFO:beir.retrieval.evaluation:MAP@3: 0.0002
INFO:beir.retrieval.evaluation:MA

OSError: [Errno 95] Operation not supported: '/content/drive/MyDrive/Contriever/train/output/checkpoint/lastlog' -> '/content/drive/MyDrive/Contriever/train/output/checkpoint/latest'

# Save the MoCo model

In [ ]:
torch.save(model1.state_dict(), "./output/moco_model_final_wiki.pth")
print("Model weights saved to ./output/moco_model_final_wiki.pth")

# Ours and ours+ce comparison

In [ ]:
import torch
from transformers import AutoTokenizer, AutoConfig
from src.contriever import Contriever
from src.beir_utils import DenseEncoderModel

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "facebook/contriever"
ckpt_path = "./output/moco_model_final_wiki.pth"

In [ ]:
#Load MoCo model weights
state_dict = torch.load(ckpt_path, map_location=device)

#Extract encoder_q.* keys
encoder_q_state_dict = {k.replace("encoder_q.", ""): v for k, v in state_dict.items() if k.startswith("encoder_q.")}

In [ ]:
# Step 4: Load into a plain Contriever
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
retriever = Contriever(config)
retriever.load_state_dict(encoder_q_state_dict)
retriever.eval()
retriever.to(device)

In [ ]:
# Step 5: Wrap for BEIR
encoder_model = DenseEncoderModel(
    query_encoder=retriever,
    doc_encoder=retriever,
    tokenizer=tokenizer,
    normalize_text=True,
    norm_query=True,
    norm_doc=True,
)

In [ ]:
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.search.dense import DenseRetrievalExactSearch
from beir.retrieval.evaluation import EvaluateRetrieval
from sentence_transformers import CrossEncoder
from beir.reranking import Rerank

In [ ]:
datasets = ["nq", "triviaqa"]
beir_root = "../../../datasets/beir"
results_bi = {}
results_rerank = {}

In [ ]:
for dataset in datasets:
    print(f"\nEvaluating on {dataset}...")
    data_path = os.path.join(beir_root, dataset)
    corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")
    corpus = dict(list(corpus.items())[:1000])

    retriever = EvaluateRetrieval(DenseRetrievalExactSearch(encoder_model, batch_size=1), score_function="dot")
    results = retriever.retrieve(corpus, queries)

    # Ours (Bi-Encoder)
    ndcg, _, _, _ = retriever.evaluate(qrels, results, retriever.k_values)
    results_bi[dataset] = ndcg

    # Ours+CE
    ce = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2", max_length=512)
    reranker = Rerank(ce)
    rerank_results = reranker.rerank(corpus, queries, results, top_k=10)
    rerank_ndcg, _, _, _ = retriever.evaluate(qrels, rerank_results, retriever.k_values)
    results_rerank[dataset] = rerank_ndcg

In [ ]:
import pandas as pd

data = {
    "Dataset": ["nq", "triviaqa"],
    "Ours (Bi-Encoder)": [
        round(results_bi["nq"]["NDCG@10"] * 100, 2),
        round(results_bi["triviaqa"]["NDCG@10"] * 100, 2)
    ],
    "Ours+CE (Cross-Encoder)": [
        round(results_rerank["nq"]["NDCG@10"] * 100, 2),
        round(results_rerank["triviaqa"]["NDCG@10"] * 100, 2)
    ],
}

df = pd.DataFrame(data).set_index("Dataset")
display(df)


# Load the MoCo model

In [ ]:
moco_model= moco.MoCo(opt).to(opt.device)
moco_model.load_state_dict(torch.load("./output/moco_model_final.pth", map_location=opt.device))

# Using the model to check document similarity

In [ ]:
moco_model.eval()
encoder= moco_model.get_encoder()
encoder.config.pooling = "average"

Function to get embeddings of the input text

In [ ]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=opt.chunk_length)
    inputs = {k: v.to(opt.device) for k, v in inputs.items()}

    with torch.no_grad():
        emb = encoder(**inputs, normalize=True)
    return emb

Measure similarity using cosine similarity

In [ ]:
import torch.nn.functional as F
doc1 = "Dog is a pet animal"
doc2 = "Cat is a pet animal"
doc3 = "Paris is the capital of France"

e1 = get_embedding(doc1)
e2 = get_embedding(doc2)
e3 = get_embedding(doc3)

print("Sim(doc1, doc2):", F.cosine_similarity(e1, e2, dim=-1).item())
print("Sim(doc1, doc3):", F.cosine_similarity(e1, e3, dim=-1).item())


In [ ]:
doc4 = "Deep learning is a sub-field of machine learning"
doc5 = "It is going to rain today"
doc6 = "Machine learning is a sub-field of artificial intelligence"

e4 = get_embedding(doc4)
e5 = get_embedding(doc5)
e6 = get_embedding(doc6)

print("Sim(doc4, doc5):", F.cosine_similarity(e4, e5, dim=-1).item())
print("Sim(doc5, doc6):", F.cosine_similarity(e5, e6, dim=-1).item())
print("Sim(doc4, doc6):", F.cosine_similarity(e4, e6, dim=-1).item())


# Misc

## Download and preprocess Wikipedia

In [ ]:
from datasets import load_dataset
ds = load_dataset("wikipedia", "20220301.en", split="train[:10000]")

In [ ]:
# Save text to a file
with open("wikipedia_subset.txt", "w", encoding="utf-8") as f:
    for article in ds:
        title = article["title"].strip()
        text = article["text"].strip()
        if text:
            f.write(f"{title} {text}\n")

In [ ]:
tokenizer_name = "bert-base-uncased"
tokenizer = transformers.AutoTokenizer.from_pretrained(tokenizer_name)
plain_text_path = "../datasets/wikipedia/wikipedia_subset.txt"
output_token_path = "../datasets/wikipedia/wiki_tokens.pt"

tokens = apply_tokenizer(plain_text_path, tokenizer)
print(f"Tokenized {tokens.size(0)} tokens from wikipedia.")

save(tokens, output_token_path)
print(f"Saved tokenized corpus to {output_token_path}")

## BERT tokenizer info

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")
print("Tokenizer vocabulary size:", len(tokenizer))
print(tokenizer.vocab_size)
print(model.tokenizer.get_vocab())
print("Embedding output dimension:", model.encoder_q.config.hidden_size)

## Download and preprocess TriviaQA
- For evaluation:
  TriviaQA- not a part of beir
  Download separately and convert into beir format

In [ ]:
import os, json
from datasets import load_dataset
from tqdm import tqdm

# Load TriviaQA dataset
dataset = load_dataset("trivia_qa", "unfiltered", split="validation")

# Output directory in BEIR format
output_dir = "../datasets/beir/triviaqa"
os.makedirs(os.path.join(output_dir, "corpus"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "qrels"), exist_ok=True)

corpus = {}
queries = {}
qrels = {}

for i, example in enumerate(tqdm(dataset)):
    qid = f"q{i}"
    queries[qid] = example["question"]
    answer = example["answer"]["value"]

    # Use the provided context as the document (for real use, replace with Wikipedia passage retrieval)
    doc_id = f"d{i}"
    context = example.get("context", "No context found")

    corpus[doc_id] = {
        "title": "",
        "text": context,
    }

    # For BEIR, treat this doc as relevant to the query
    qrels[qid] = {doc_id: 1}

# Write BEIR-style files
with open(os.path.join(output_dir, "corpus.jsonl"), "w") as f:
    for doc_id, doc in corpus.items():
        f.write(json.dumps({"_id": doc_id, **doc}) + "\n")

with open(os.path.join(output_dir, "queries.jsonl"), "w") as f:
    for qid, query in queries.items():
        f.write(json.dumps({"_id": qid, "text": query}) + "\n")

with open(os.path.join(output_dir, "qrels", "test.tsv"), "w") as f:
    f.write("query-id\tcorpus-id\tscore\n")
    for qid, doc_dict in qrels.items():
        for doc_id, score in doc_dict.items():
            f.write(f"{qid}\t{doc_id}\t{score}\n")
